# parPE Lucarelli example model

In [1]:
import amici
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
sbml_file = 'lucarelli_12.xml'
model_name = 'lucarelli_12'
model_output_dir = 'lucarelli_12'

In [3]:
# Show model info
import libsbml
SBMLreader = libsbml.SBMLReader()
sbml_doc = SBMLreader.readSBML(sbml_file)
sbml_model = sbml_doc.getModel()

print('Species: ', [s.getId() for s in sbml_model.getListOfSpecies()])

print('Parameters: ', [p.getId() for p in sbml_model.getListOfParameters()])

print('\nReactions:')
for reaction in sbml_model.getListOfReactions():
    reactants = ' + '.join(['%s %s'%(int(r.getStoichiometry()) if r.getStoichiometry() > 1 else '', r.getSpecies()) for r in reaction.getListOfReactants()])
    products  = ' + '.join(['%s %s'%(int(r.getStoichiometry()) if r.getStoichiometry() > 1 else '', r.getSpecies()) for r in reaction.getListOfProducts()])
    reversible = '<' if reaction.getReversible() else ''
    print('%3s: %10s %1s->%10s\t\t[%s]' % (reaction.getId(), 
                        reactants,
                        reversible,
                        products,
                        libsbml.formulaToL3String(reaction.getKineticLaw().getMath())))


Species:  ['TGFb', 'Rec', 'TGFb_pRec', 'S2', 'S3', 'S4', 'S2_S4_S4', 'ppS2_ppS2_ppS2', 'ppS3_ppS3_ppS3', 'S4_S4_S4', 'pS2', 'pS3', 'ppS2', 'ppS3', 'ppS2_ppS2_S4', 'ppS2_ppS2_ppS3', 'ppS2_ppS3_ppS3', 'ppS3_ppS3_S4', 'ppS2_ppS3_S4', 'ppS3_S4_S4', 'ppS2_S4_S4', 'geneA', 'geneB', 'geneC', 'geneD', 'geneE', 'geneF', 'geneG', 'geneH', 'geneI', 'geneJ', 'geneK', 'geneL']
Parameters:  ['Rec_act', 'S_dephos', 'S_dephosphos', 'S_phos', 'geneA_act1', 'geneA_act2', 'geneA_act3', 'geneA_inh1', 'geneA_inh2', 'geneA_inh3', 'geneA_turn', 'geneB_act1', 'geneB_act2', 'geneB_act3', 'geneB_inh1', 'geneB_inh2', 'geneB_inh3', 'geneB_turn', 'geneC_act1', 'geneC_act2', 'geneC_act3', 'geneC_inh1', 'geneC_inh2', 'geneC_inh3', 'geneC_turn', 'geneD_act1', 'geneD_act2', 'geneD_act3', 'geneD_inh1', 'geneD_inh2', 'geneD_inh3', 'geneD_turn', 'geneE_act1', 'geneE_act2', 'geneE_act3', 'geneE_inh1', 'geneE_inh2', 'geneE_inh3', 'geneE_turn', 'geneF_act1', 'geneF_act2', 'geneF_act3', 'geneF_inh1', 'geneF_inh2', 'geneF_inh

## Generate model code

In [4]:
# remove time from the list of names
observables_list = ['observable_Ski', 'observable_Skil', 'observable_Dnmt3a', 'observable_Sox4', 'observable_Jun', 'observable_Smad7', 'observable_Klf10', 'observable_Bmp4', 'observable_Cxcl15', 'observable_Dusp5', 'observable_Tgfa', 'observable_Pdk4'] #
fixed_parameters = ['init_TGFb', 'init_S2', 'init_S3', 'init_S4']

def createModule(sbml_model_file, model_name, model_output_dir):
    """Create Python module from SBML model"""
    sbmlImporter = amici.SbmlImporter(sbml_model_file)
    sbml = sbmlImporter.sbml
    global observables
    observables = amici.assignmentRules2observables(
                  sbmlImporter.sbml, # the libsbml model object
                  filter=lambda variableId: variableId in observables_list)

    sbmlImporter.sbml2amici(model_name,
                            output_dir=model_output_dir,
                            observables=observables,
                            constantParameters=fixed_parameters)


In [ ]:
createModule(sbml_file, model_name, model_output_dir)

## Test simulation

In [ ]:
def plotStateTrajectories(rdata):
    for ix in range(22,33):
        plt.plot(rdata['t'], rdata['x'][:, ix], label='$x_%d$' % ix)
        plt.xlabel('$t$ (s)')
        plt.ylabel('$x_i(t)$ (mmol/ml)')
        plt.legend()
        plt.title('State trajectories')
    plt.show()
    
def plotObservableTrajectories(rdata):
    for iy in range(rdata['y'].shape[1]):
        plt.plot(rdata['t'], rdata['y'][:, iy], label='$y_%d$' % iy)
        plt.xlabel('$t$ (s)')
        plt.ylabel('$y_i(t)$ (AU)')
        plt.legend()
        plt.title('Observables')
    
    plt.show()


In [ ]:
# load model
sys.path.insert(0, model_output_dir)
import lucarelli_12 as modelModule

model = modelModule.getModel()
model.setTimepoints(amici.DoubleVector(np.linspace(0, 250, 25))) 
solver = model.getSolver()
rdata = amici.runAmiciSimulation(model, solver)

default_parameters = np.array(model.getParameters())
print(default_parameters)
print(rdata)

plotStateTrajectories(rdata)
plotObservableTrajectories(rdata)


## Create in silico experimental data for parameter estimation

- Simulate time-course for four different conditions
- Add gaussian noise according to selected sigma parameter
- Mimic 2 experimental batches: odd-numbered condition indices and even-numbered conditions have different offset parameter

In [ ]:
from pandas import DataFrame

def createConditionDataframe(indices, conditions, parameters):
    """Create dataframe with fixed-parameters for each condition to simulate"""
    conditionDf = pd.DataFrame(index=indices)
    conditionDf['ID'] = conditionDf.index
    for icondition, condition in enumerate(conditions):
        conditionDf['condition-%d' % icondition] = condition
    
    return conditionDf

def getReturnDataForCondition(model, solver, condition, simulationParameters, sigmay):
    model.setParameters(amici.DoubleVector(simulationParameters))
    
    # simulate without measurements
    edata = amici.ExpData(model.get())
    edata.fixedParameters = amici.DoubleVector(condition)
    edata.my = amici.DoubleVector(np.full(shape=model.nt() * model.nytrue, fill_value=np.nan))
    rdata = amici.runAmiciSimulation(model, solver, edata)        
    # fixedParametersPreequilibration = 

    # confirm gradient is 0 for real measurements and save expected llh
    measurement = rdata['y']
    measurement = np.random.normal(loc=rdata['y'], scale=sigmay)
    measurement = np.random.normal(
        loc=rdata['y'], 
        scale=sigmay[0][0])
    # print(measurement)

    edata.my = amici.DoubleVector(measurement.flatten())
    edata.sigmay = amici.DoubleVector(sigmay.flatten())
    model.requireSensitivitiesForAllParameters()
    rdata = amici.runAmiciSimulation(model, solver, edata)
    # return generated noisy measurents
    rdata['y'] = measurement
    return rdata

expectedLlh = 0.0
sigma_default = 0.1 # parameters are lin
timepoints = np.linspace(0, 100, 5)

model = modelModule.getModel()
true_parameters = np.array(model.getParameters())

# setup model & solver
model = modelModule.getModel()
model.setTimepoints(amici.DoubleVector(timepoints))
model.setParameters(amici.DoubleVector(true_parameters))

solver = model.getSolver()
solver.setMaxSteps(10000)

# generate conditon-vectors
sigma = 0.5
mu = 1.5
nConditions = 10
init_TGFb = np.linspace(0,1,11)
init_conditions = [10.0**(mu + sigma * np.random.randn(4))]
init_conditions[0][0] = init_TGFb[np.random.randint(0,11)]
for i in range(1,nConditions):
    init_conditions.append(np.array(10.0**(mu + sigma * np.random.randn(4))))
    init_conditions[i][0] = init_TGFb[np.random.randint(0,11)]
    
indices = fixed_parameters
conditionDf = createConditionDataframe(indices, init_conditions, true_parameters)
    
df = pd.DataFrame(data={
    'observable': [],
    'condition': [],
    'conditionRef': [],
    'time': [],
    'measurement': [],
})

for icondition, condition in enumerate(init_conditions):      
    
    simulationParameters = true_parameters
    sigmay = np.ones(shape=(model.nt(), model.nytrue)) * sigma_default
    
    # simulate condition
    rdata = getReturnDataForCondition(model, solver, condition, simulationParameters, sigmay)

    expectedLlh += rdata['llh']
    
    conditionName = 'condition-%d' % icondition
    
    # Append data
    for iy, observableName in enumerate(observables.keys()):
        scalingParameter = [''] 
        sigma = sigmay[:, iy]
        
        df = df.append(pd.DataFrame(
        {'observable'      : [observableName] * model.nt(),
         'condition'       : [conditionName]  * model.nt(),
         'conditionRef'    : ['']  * model.nt(),
         'scalingParameter': scalingParameter * model.nt(),
         'time'            : np.array(model.getTimepoints()),
         'measurement'     : rdata['y'][:, iy],
         'sigma'           : sigma
         }), ignore_index=True)
print('Expected llh: ', expectedLlh)

In [ ]:
conditionDf

In [ ]:
df

In [ ]:
# write data frames to file
measurement_file = 'example_data.tsv'
fixed_parameter_file = 'example_data_fixed.tsv'
hdf5File = 'example_data.h5'

df.to_csv(measurement_file, sep='\t', index=False)
conditionDf.to_csv(fixed_parameter_file, sep='\t', index=False)

! /bin/bash -c "if [[ -f example_data.h5 ]]; then cp example_data.h5 example_data.h5.bak; fi"
# convert to HDF5
import subprocess
out = subprocess.run(['%s/generateHDF5DataFileFromText.py' % os.path.join(os.getcwd(), '..', '..', '..', 'misc'),
    hdf5File,
    sbml_file,
    model_output_dir,
    measurement_file,
    fixed_parameter_file], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
print(out.stdout.decode("utf-8"))
print(out.returncode)


In [ ]:
# write true parameters as first starting point, an perturbed additional points
# two times the same point to check for reproducibility
import h5py
with h5py.File(hdf5File, 'r+') as f:
    pscale = f['/parameters/pscale'][:]
    true_parameters_scaled = true_parameters.copy()
    for i, p in enumerate(pscale):
        if p == 2:
            if true_parameters[i] < 1e-10:
                true_parameters_scaled[i] = -10.0
            else:
                true_parameters_scaled[i] = np.log10(true_parameters[i])
    
    print(true_parameters_scaled)
    
    for i in range(10):
        parameters = true_parameters_scaled
        #parameters = parameters + np.random.normal(0.0, 0.2 + i * 0.1, true_parameters.shape)
        #parameters = np.random.uniform(-3, 5, true_parameters.shape)
        
        #print(parameters)
        f['/optimizationOptions/randomStarts'][:,2*i]   = parameters
        f['/optimizationOptions/randomStarts'][:,2*i+1] = parameters


In [ ]:
# enable derivate checker
#!../../../misc/optimizationOptions.py example_data.h5 -s ipopt/derivative_test first-order
#!../../../misc/optimizationOptions.py example_data.h5 -s ipopt/derivative_test_print_all yes
!../../../misc/optimizationOptions.py example_data.h5 -s numStarts 5
!../../../misc/optimizationOptions.py example_data.h5 -s hierarchicalOptimization 0
!../../../misc/optimizationOptions.py example_data.h5


## Gradient check

## Optimize

In [ ]:
!pwd
!(cd ../../../build && exec make -j12)
!../../../build/amici/examples/lucarelli/example_lucarelli_multi -o deleteme/ example_data.h5


In [ ]:
for i in range(3):
    !PARPE_NO_DEBUG=1 ../../../build/amici/examples/lucarelli/example_lucarelli_multi -t gradient_check -o deletemegc/ example_data.h5


In [ ]:
filename = 'deleteme/_rank-0001.h5'
with h5py.File(filename, 'r') as f:
    trajectory = f['/multistarts/0/iterCostFunCost'][:]
for start in range(trajectory.shape[0]):
    plt.plot(trajectory[start])
trajectory

## Compare results

In [ ]:
def compareResults(filename, expectedNllh):
    with h5py.File(filename, 'r') as f:
        final_parameters = f['/multistarts/0/finalParameters'][:]
        exit_status = f['/multistarts/0/exitStatus'][:]
        final_cost = f['/multistarts/0/finalCost'][:]

    for i, p in enumerate(pscale):
        if p == 2:
            final_parameters[i] = np.power(10, final_parameters[i])

    print("#  __Exp___ __Act___ __Err___ _RelErr___")
    for i in range(len(true_parameters)):
        error = final_parameters[i]-true_parameters[i]
        rel_error = error / true_parameters[i]
        print('%d: %6f %6f %6f %6f' % (i, true_parameters[i], 
                                       final_parameters[i], 
                                       error,rel_error))
    print('Status: %d' % exit_status)
    print('Cost: %f (expected: %f)' % (final_cost, expectedNllh))

In [ ]:
compareResults('deleteme/_rank-0001.h5', -expectedLlh)

In [ ]:
!pwd

In [ ]:
!rm sim.h5
!../../../build/amici/examples/lucarelli/example_lucarelli_multi_simulator deleteme/_rank-0001.h5 / sim.h5 / --at-optimum

In [ ]:
with h5py.File('sim.h5', 'r') as f:
    ysim = f['/multistarts/0/ySim'][:]
    ymes = f['/multistarts/0/yMes'][:]
print(ysim.shape)
print(ymes.shape)

In [ ]:
def plotCorrelation(ymes, ysim):
    for iy in range(ysim.shape[2]):
        fig, ax = plt.subplots()
        for icondition in range(ysim.shape[0]):
            x = ymes[icondition, :, iy]
            y = ysim[icondition, :, iy]
            print(x, y)
            ax.scatter(x, y, label='$y_%d$ condition %d' % (iy, icondition))
        plt.xlabel('measurement (AU)')
        plt.ylabel('simulation (AU)')
        plt.gca().set_aspect('equal', adjustable='box')
        plt.title('Observable %d' % iy)
        plt.legend()
        plt.show()
        
plotCorrelation(ymes, ysim)

In [ ]:
def plotTrajectoryFit(ymes, ysim, timepoints):
    for icondition in range(ysim.shape[0]):
        fig, ax = plt.subplots()
        for iy in range(ysim.shape[2]):
            ax.plot(timepoints, ysim[icondition, :, iy], label='$y_%d$ sim' % (iy), alpha=0.7, c='C%d'%iy)
            ax.plot(timepoints, ymes[icondition, :, iy], label='$y_%d$ mes' % (iy), linestyle='dotted', c='C%d'%iy)
        plt.xlabel('$t$ (s)')
        plt.ylabel('$y_i(t)$ (AU)')
        plt.title('Condition %d' %icondition)
        plt.legend()
        plt.show()

plotTrajectoryFit(ymes, ysim, timepoints)